In [1]:
import os, random, json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler

import optuna
from metrics import metrics_from_preds
from preprocess import *


# Carga de datos

In [2]:
def set_seed(seed=42):
    SEED = seed
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    os.environ["PYTHONHASHSEED"] = str(seed)

In [3]:
set_seed(42)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando dispositivo:", device)

Usando dispositivo: cpu


In [5]:
vids_training, labels_training, vids_val, labels_val, vids_test, labels_test = load_data()

In [6]:
class VideoDataset(Dataset):
    def __init__(self, videos, labels):
        self.videos = videos
        self.labels = labels

    def __len__(self):
        return len(self.videos)

    def __getitem__(self, idx):
        x = torch.tensor(np.array(self.videos[idx]), dtype=torch.float32)
        y = torch.tensor(self.labels[idx], dtype=torch.long)
        return x, y

In [7]:
set_seed()

train_dataset = VideoDataset(vids_training, labels_training)
val_dataset   = VideoDataset(vids_val, labels_val)

g = torch.Generator().manual_seed(42)
perm = torch.randperm(len(train_dataset), generator=g).tolist()
train_sampler = SubsetRandomSampler(perm)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False, num_workers=0, sampler=train_sampler)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)

# Modelo SAN

In [8]:
class SelfAttentionBlock(nn.Module):
    def __init__(self, embed_dim=258, num_heads=4, dropout=0.1):
        super().__init__()
        self.attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout, batch_first=True)
        self.ff = nn.Sequential(
            nn.Linear(embed_dim, embed_dim),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)

    def forward(self, x):
        attn_out, _ = self.attn(x, x, x)
        x = self.norm1(x + attn_out)
        ff_out = self.ff(x)
        x = self.norm2(x + ff_out)
        return x

In [9]:
class SANModel(nn.Module):
    def __init__(self, in_features=258, num_classes=4, num_heads=4, layers=2, hidden=128, dropout=0.1):
        super().__init__()
        self.input_proj = nn.Linear(in_features, hidden)
        self.layers = nn.ModuleList([
            SelfAttentionBlock(embed_dim=hidden, num_heads=num_heads, dropout=dropout) for _ in range(layers)
        ])
        self.head = nn.Linear(hidden, num_classes)

    def forward(self, x):
        x = self.input_proj(x)
        for layer in self.layers:
            x = layer(x)
        x = x.mean(dim=1)
        return self.head(x)

# Optimización de Hiperparametros

In [10]:
def optimize_san(train_loader, val_loader, n_trials=50, max_epochs=80, save_prefix="hpo_san"):
    set_seed()
    sampler = optuna.samplers.TPESampler(seed=42)
    study = optuna.create_study(direction="maximize", sampler=sampler, pruner=optuna.pruners.MedianPruner())
    all_trials = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def objective(trial):
        hidden  = trial.suggest_categorical("hidden",[96,128,160,192])
        layers  = trial.suggest_int("layers", 2, 5)
        num_heads = trial.suggest_categorical("num_heads", [2, 4, 8])
        drop    = trial.suggest_float("dropout", 0.0, 0.5)
        lr      = trial.suggest_float("lr", 5e-5, 5e-3, log=True)
        wd      = trial.suggest_float("weight_decay", 1e-10, 1e-3, log=True)

        model = SANModel(hidden=hidden, layers=layers, num_heads=num_heads, dropout=drop).to(device)
        crit = nn.CrossEntropyLoss()
        opt  = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)

        best_val = 0.0
        for epoch in range(max_epochs):
            model.train()
            for X, y in train_loader:
                X, y = X.to(device), y.to(device)
                opt.zero_grad()
                out = model(X)
                loss = crit(out, y)
                loss.backward(); opt.step()
            model.eval()
            y_true, y_pred = [], []
            with torch.no_grad():
                for X, y in val_loader:
                    X, y = X.to(device), y.to(device)
                    pred = model(X).argmax(1)
                    y_true.extend(y.cpu().numpy()); y_pred.extend(pred.cpu().numpy())
            val_acc = metrics_from_preds(y_true, y_pred)["accuracy"]
            if val_acc > best_val: best_val = val_acc
            trial.report(val_acc, epoch)
            if trial.should_prune(): raise optuna.TrialPruned()

        all_trials.append({"trial": trial.number, "params": trial.params, "best_val_acc": best_val})
        return best_val

    study.optimize(objective, n_trials=n_trials)
    with open(f"{save_prefix}_all.json","w") as f: json.dump(all_trials,f,indent=2)
    with open(f"{save_prefix}_best.json", "w") as f: json.dump({"best_params":study.best_params, "best_value":study.best_value}, f, indent=2)
    return study

In [11]:
study_san = optimize_san(train_loader, val_loader, n_trials=40, max_epochs=100)

[I 2025-11-03 13:12:09,923] A new study created in memory with name: no-name-62e59cfe-86ed-456a-82b2-122c93a7526c
[I 2025-11-03 13:13:23,031] Trial 0 finished with value: 0.9333333333333333 and parameters: {'hidden': 128, 'layers': 2, 'num_heads': 8, 'dropout': 0.3005575058716044, 'lr': 0.0013035123791853842, 'weight_decay': 1.3934502251337584e-10}. Best is trial 0 with value: 0.9333333333333333.
[I 2025-11-03 13:14:13,537] Trial 1 finished with value: 0.9277777777777778 and parameters: {'hidden': 96, 'layers': 2, 'num_heads': 4, 'dropout': 0.14561457009902096, 'lr': 0.0008369042894376068, 'weight_decay': 9.472334467618544e-10}. Best is trial 0 with value: 0.9333333333333333.
[I 2025-11-03 13:15:48,929] Trial 2 finished with value: 0.9277777777777778 and parameters: {'hidden': 192, 'layers': 2, 'num_heads': 4, 'dropout': 0.3037724259507192, 'lr': 0.0001096524277832185, 'weight_decay': 2.853390105240223e-10}. Best is trial 0 with value: 0.9333333333333333.
[I 2025-11-03 14:34:14,148] Tr

In [12]:
print("Mejor resultado:", study_san.best_value)
print("Mejores hiperparámetros:", study_san.best_params)

Mejor resultado: 0.95
Mejores hiperparámetros: {'hidden': 96, 'layers': 5, 'num_heads': 4, 'dropout': 0.26136641469099703, 'lr': 0.00035813934999486707, 'weight_decay': 1.5063777323554432e-10}
